In [ ]:
!pip install --quiet litellm
!pip install --quiet crewai
!pip install --quiet crewai-tools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 65.9 MB/s eta 0:00:

In [ ]:
from crewai import Agent, Task, Crew, Process, LLM
from crewai_tools import SerperDevTool, ScrapeWebsiteTool
from crewai.tools import BaseTool
from litellm import completion
import os
from google.colab import userdata
from pydantic import BaseModel, Field
from typing import List, Optional

In [ ]:
google_api_key = userdata.get('GOOGLE_API_KEY')
os.environ["GEMINI_API_KEY"] = userdata.get('GOOGLE_API_KEY')

llm = LLM(
    model="gemini/gemini-1.5-flash",
    temperature=0.7,
)

In [ ]:
class GameDeveloperAgent():
  def senior_engineer_agent(self)->Agent:
    return Agent(
        role='Senior Software Engineer',
        goal='Create software as needed',
        backstory='You are a Senior Software Engineer at a leading tech think tank.\
        Your expertise in programming in python. and do your best to produce perfect code',
        llm=llm
    )

  def qa_engineer_agent(self)->Agent:
    return Agent(
        role='Software Quality Control Engineer',
        goal='Create Perfect code, by analyzing the code that is given for errors',
        backstory='You are a software engineer that specializes in checking code\
        for errors. You have an eye for detail and a knack for finding\
        hidden bugs. You check for missing imports, variable declarations, mismatched\
        brackets and syntax errors. You also check for security vulnerabilities, and logic error',
        llm=llm
    )

  def chief_qa_engineer_agent(self)->Agent:
    return Agent(
        role='Chief Software Quality Control Engineer',
        goal='Ensure that the code does the job that it is supposed to do',
        backstory='You feel that programmers always do only half the job, so you are\
          super dedicate to make high quality code.',
        llm=llm,
        allow_delegation=True
      )

In [ ]:
class GameDeveloperTask():
  def code_task(self, agent)->Task:
    return Task(
        description="You will create a game using python, these are the instructions:\
        Instructions\
        ------------\
        {game}",
        agent=agent,
        expected_output="Your Final answer must be the full python code, only the python code and nothing else.",
    )

  def review_task(self, agent)->Task:
    return Task(
      description="You will create a game using python, these are the instructions::\
      Instructions\
      ------------\
      {game}\
      Using the code you got, check for errors. Check for logic errors,\
      syntax errors, missing imports, variable declarations, mismatched brackets,\
      and security vulnerabilities",
      agent=agent,
      expected_output="Your Final answer must be the full python code, only the python code and nothing else.",
    )

  def evaluate_task(self, agent)->Task:
    return Task(
      description="You will create a game using python, these are the instructions::\
      Instructions\
      ------------\
      {game}\
      You will look over the code to insure that it is complete and\
      does the job that it is supposed to do.",
      agent=agent,
      expected_output="Your Final answer must be the full python code, only the python code and nothing else.",
    )

In [ ]:
agents = GameDeveloperAgent()
tasks = GameDeveloperTask()

senior_engineer = agents.senior_engineer_agent()
qa_engineer = agents.qa_engineer_agent()
chief_qa_engineer = agents.chief_qa_engineer_agent()

code_task = tasks.code_task(senior_engineer)
review_task = tasks.review_task(qa_engineer)
evaluate_task = tasks.evaluate_task(chief_qa_engineer)

crew = Crew(
    agents=[senior_engineer, qa_engineer, chief_qa_engineer],
    tasks=[code_task, review_task, evaluate_task],
    process=Process.sequential,
    verbose=True
)

In [ ]:
print("## Welcome to the Game Crew")
print('-------------------------------')
game = """
    Game Overview:

    Pac-Man is a classic arcade game where the player controls a character, Pac-Man, through a maze. The objective is to eat all the pellets in the maze while avoiding four ghosts that pursue Pac-Man. If Pac-Man eats a large power pellet, the ghosts turn blue, and Pac-Man can eat them for extra points.
    The game is over when Pac-Man is caught by a ghost or when the player runs out of lives.
    Core Game Elements:

        Maze Layout:
            The game is set within a grid-like maze. The walls are solid, and Pac-Man cannot pass through them.
            The maze contains corridors where Pac-Man can move in four directions: up, down, left, and right. Some sections of the maze loop back on themselves.
            The maze contains two types of special tiles:
                Pellets: Small dots scattered throughout the maze, worth 10 points each.
                Power Pellets: Larger dots placed in the four corners of the maze, worth 50 points each. Eating a Power Pellet allows Pac-Man to eat the ghosts for a limited time.

        Player Controls:
            The player controls Pac-Man's movement using arrow keys (or other directional inputs, such as WASD).
            Pac-Man moves continuously in the chosen direction until blocked by a wall or until the player changes direction.
            Pac-Man cannot stop moving, so the player must carefully time movements and changes in direction.

        Pac-Man Mechanics:
            Movement: Pac-Man moves one tile at a time in a grid-based movement system.
            Collision Detection: Pac-Man collides with walls, pellets, power pellets, and ghosts. Pac-Man cannot pass through walls.
            Pellet Collection: When Pac-Man moves onto a tile containing a pellet, it is "eaten," and the pellet disappears.
            Power Pellet Effects: Eating a Power Pellet allows Pac-Man to turn the ghosts blue for a short period (usually 7-10 seconds). During this time, Pac-Man can eat the ghosts for extra points. Ghosts revert to their regular form after the time limit.

        Ghosts:
            There are four ghosts: Blinky, Pinky, Inky, and Clyde. Each has a distinct behavior pattern:
                Blinky (Red Ghost): Aggressively pursues Pac-Man, always targeting his current location.
                Pinky (Pink Ghost): Attempts to ambush Pac-Man by aiming four tiles ahead of Pac-Man's current direction.
                Inky (Cyan Ghost): Has a more complex behavior, targeting an area between Pac-Man and Blinky's current location.
                Clyde (Orange Ghost): Alternates between chasing Pac-Man and wandering randomly when he gets too close to Pac-Man.
            Ghost Movement: Ghosts move one tile at a time, just like Pac-Man, and they can change directions at intersections. Their goal is to catch Pac-Man.
            Ghost States:
                Chase Mode: Ghosts actively pursue Pac-Man based on their unique behavior patterns.
                Scatter Mode: Ghosts move to specific corners of the maze, where they “scatter” and remain for a brief period before returning to Chase mode.
                Frightened Mode: After Pac-Man eats a Power Pellet, ghosts turn blue and flee from Pac-Man. In this mode, Pac-Man can eat them for extra points. When a ghost is eaten, it respawns at the center of the maze and resumes chasing Pac-Man.

        Scoring System:
            Eating a pellet: 10 points.
            Eating a Power Pellet: 50 points.
            Eating a ghost (in Frightened mode):
                First ghost: 200 points.
                Second ghost: 400 points.
                Third ghost: 800 points.
                Fourth ghost: 1600 points.
            Clearing a level (eating all the pellets): Bonus points for completing the level.

        Lives and Game Over:
            Pac-Man starts the game with 3 lives.
            If a ghost touches Pac-Man while in its normal or chase mode, Pac-Man loses a life.
            When all lives are lost, the game ends.

        Level Progression:
            After all pellets and Power Pellets are consumed in the maze, Pac-Man progresses to the next level.
            Each new level increases the game difficulty, making the ghosts move faster.
            At higher levels, the time that ghosts remain blue after eating a Power Pellet decreases, eventually reaching a point where they no longer turn blue.

        Warp Tunnels:
            The maze has two special tunnels on the left and right edges that act as "warp tunnels."
            When Pac-Man or the ghosts enter one side, they instantly reappear on the opposite side of the maze.

    Mechanics Used in the Game:

        Tile-Based Movement:
            The entire game operates on a grid, where each movement happens from one tile to another. Both Pac-Man and the ghosts must follow the grid's structure.

        Pathfinding (Ghost AI):
            The ghosts use basic pathfinding algorithms to chase Pac-Man. One common method for this is the A algorithm* or a simplified greedy algorithm to determine the shortest path toward Pac-Man.
            Each ghost has its unique targeting behavior, ranging from direct pursuit to attempting ambush strategies.

        State Management:
            Pac-Man's State: Handles whether Pac-Man is in a normal state, Power Pellet state (can eat ghosts), or has collided with a ghost.
            Ghosts' State: Manages transitions between three ghost states:
                Chase State: Actively chasing Pac-Man.
                Scatter State: Retreats to their designated corners.
                Frightened State: Turns blue and flees from Pac-Man, allowing Pac-Man to eat them.

        Collision Detection:
            Pellets and Pac-Man: When Pac-Man's position matches a pellet's position, the pellet is eaten.
            Ghosts and Pac-Man: When Pac-Man's position matches a ghost's position:
                If the ghost is in Frightened mode, Pac-Man eats the ghost.
                If the ghost is in Chase or Scatter mode, Pac-Man loses a life.

        Timer and Speed Control:
            The game runs on a time-based loop where Pac-Man and the ghosts move at set intervals.
            Ghosts' speeds increase over time, making higher levels more difficult.

        Level Design and Randomness:
            While the layout of the maze stays the same, the randomness in ghost behavior and increasing speed add variability to each playthrough.

        Game Over Conditions:
            When Pac-Man has no remaining lives, the game ends, displaying a "Game Over" screen.
            The players final score is shown"""

inputs={
    'game': game
}

game_crew = crew.kickoff(inputs=inputs)
print("\n\n########################")
print("## Here is the result")
print("########################\n")
print("final code for the game:")
print(game)

## Welcome to the Game Crew
-------------------------------
# Agent: Senior Software Engineer
## Task: You will create a game using python, these are the instructions:        Instructions        ------------        
    Game Overview:

    Pac-Man is a classic arcade game where the player controls a character, Pac-Man, through a maze. The objective is to eat all the pellets in the maze while avoiding four ghosts that pursue Pac-Man. If Pac-Man eats a large power pellet, the ghosts turn blue, and Pac-Man can eat them for extra points. 
    The game is over when Pac-Man is caught by a ghost or when the player runs out of lives.
    Core Game Elements:

        Maze Layout:
            The game is set within a grid-like maze. The walls are solid, and Pac-Man cannot pass through them.
            The maze contains corridors where Pac-Man can move in four directions: up, down, left, and right. Some sections of the maze loop back on themselves.
            The maze contains two types of spec